In [135]:
#Permanent cell 1
import arcpy
import pandas as pd
import sqlite3
import math
import numpy as np
import os
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side

In [4]:
#Permanent cell 2
def sql_to_df(sql,model):
    con = sqlite3.connect(model)
    df = pd.read_sql(sql, con)
    con.close()
    return df

def execute_sql(sqls,model):
    con = sqlite3.connect(model)
    cur = con.cursor()
    if type(sqls) == list:
        for sql in sqls:
            cur.execute(sql)
    else:         
        cur.execute(sqls)
    cur.close()
    con.commit()
    con.close()

In [5]:
#Permanent cell 3
# User Input, to move to separate sheet so no permanent cell
#stop trace if more pipes than max_steps traced from catchment, must be an endless loop. 
max_steps = 1000 

update_field_in_model = True
update_field = 'Description'

output_folder = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\Rawn_Tool\Output"
model_path = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\NSSA_Base_2018pop.sqlite"
sewer_area = 'NSSA'
pop_book = r"\\prdsynfile01\LWS_Modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\MPF4_Temp_Hold\NSSA_Master_Population_File_4_No_2237_ResArea.xlsx"
pop_sheet = 'MPF Update 4'
model = 'NSSA'
gdb_name = 'RAWN.gdb'
gdb_name_dissolve = 'RAWN_Dissolve.gdb' #To keep clutter out of main database

#Options to skip time consuming steps during debug, must be True during production runs
run_dissolve = False
run_dissolve_append = False
run_jpg = True
run_import = False
run_html = False


In [6]:
#Permanent cell 4
#Set up column names

years = [2060,2070,2080,2090,2100]
categories = ['res','com','ind','inst','infl','infi']

mpf_col_dict = {}

area_col_dict = {}
area_col_dict['res'] = 'Area_Res'
area_col_dict['com'] = 'Area_Com'
area_col_dict['ind'] = 'Area_Ind'
area_col_dict['inst'] = 'Area_Inst'
area_col_dict['ini'] = 'Area_Total'

per_unit_dict = {}
per_unit_dict['res'] = 320
per_unit_dict['com'] = 33700 
per_unit_dict['ind'] = 56200
per_unit_dict['inst'] = 33700
per_unit_dict['infl'] = 5600
per_unit_dict['infi'] = 5600

unit_dict = {}
unit_dict['res'] = 'L/c/d'
unit_dict['com'] = 'L/ha/d'
unit_dict['ind'] = 'L/ha/d'
unit_dict['inst'] = 'L/ha/d'
unit_dict['infl'] = 'L/ha/d'


header_dict = {}
# header_dict['gen'] = ['GENERAL INFO',['TYPE','MODELID','CATCHMENT','ID','YEAR','LOCATION']]
header_dict['gen'] = ['GENERAL INFO',['TYPE','CATCHMENT','YEAR','LOCATION']]
header_dict['res'] = ['RESIDENTIAL',['AREA (Ha)','POPULATION','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
header_dict['com'] = ['COMMERCIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
header_dict['ind'] = ['INDUSTRIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
header_dict['inst'] = ['INSTITUTIONAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
header_dict['ini'] = ['INFLOW / INFILTRATION',['AREA (Ha)','INFLOW (L/s)','INFILTRATION (L/s)']]
header_dict['flow'] = ['FLOWS',['AVG. SAN. FLOW (L/s)','ADWF (L/s)','PWWF (L/s)']]

avg_calc_dict = {}
avg_calc_dict['res'] = ['RESIDENTIAL','POPULATION','AVG. FLOW (L/s)']
avg_calc_dict['com'] = ['COMMERCIAL','AREA (Ha)','AVG. FLOW (L/s)']
avg_calc_dict['ind'] = ['INDUSTRIAL','AREA (Ha)','AVG. FLOW (L/s)']
avg_calc_dict['inst'] = ['INSTITUTIONAL','AREA (Ha)','AVG. FLOW (L/s)']
avg_calc_dict['infl'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFLOW (L/s)']
avg_calc_dict['infi'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFILTRATION (L/s)']

header_tuples = []
for header in header_dict:
    for sub_header in (header_dict[header][1]):
        header_tuples.append((header_dict[header][0],sub_header))
header_tuples

# columns_multiindex = pd.MultiIndex.from_tuples(header_tuples,names=['Category', 'Subcategory'])
columns_multiindex = pd.MultiIndex.from_tuples(header_tuples)
df_template = pd.DataFrame(columns=columns_multiindex)

info_list = []
for item in unit_dict:
    info_list.append([avg_calc_dict[item][0],per_unit_dict[item],unit_dict[item]])
info_df = pd.DataFrame(info_list,columns=['DESCRIPTION','AVG. FLOW','UNITS'])
info_df.set_index('DESCRIPTION',inplace=True)

In [7]:
#Permanent cell 5
#Import population
pop_df = pd.read_excel(pop_book,sheet_name=pop_sheet,dtype={'Catchment': str})#[['Catchment','Year','Pop_Total']]
pop_df.rename(columns={"Pop_Total": "Population"},inplace=True)
pop_df = pop_df[['Catchment','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Population','Area_ResLD','Area_ResHD','Area_Mixed','Area_Com','Area_Ind','Area_Inst']]
pop_df['Area_Res'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed
pop_df['Area_Total'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed + pop_df.Area_Com + pop_df.Area_Ind + pop_df.Area_Inst
pop_df['Population_Sum_Check'] = pop_df.Pop_ResLD + pop_df.Pop_ResHD + pop_df.Pop_Mixed
pop_sum_total_col = int(pop_df.Population.sum())
pop_sum_sub_cols = int(pop_df.Pop_ResLD.sum() + pop_df.Pop_ResHD.sum() + pop_df.Pop_Mixed.sum())
pop_df['Key'] = sewer_area + '@' + pop_df.Catchment + '@' + pop_df['Year'].astype(str)
pop_df.set_index('Key',inplace=True)

if pop_sum_total_col != pop_sum_sub_cols:
      raise ValueError("Error. The sum of 'Population' (" + str(pop_sum_total_col) + ") is different than the sum of 'Pop_ResLD' + 'Pop_ResHD' + 'Pop_Mixed' (" + str(pop_sum_sub_cols) + ")") 


In [8]:
#Permanent cell 6
#Import model data

node_types = {}
node_types[1] = 'Manhole'
node_types[2] = 'Basin'
node_types[3] = 'Outlet'
node_types[4] = 'Junction'
node_types[5] = 'Soakaway'
node_types[6] = 'River Junction'

sql = "SELECT catchid AS Catchment, nodeid AS Connected_Node FROM msm_Catchcon WHERE Active = 1"
catchments = sql_to_df(sql,model_path)

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], uplevel AS Outlet_Level FROM msm_Link WHERE Active = 1"
lines = sql_to_df(sql,model_path)

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Orifice WHERE Active = 1"
orifices = sql_to_df(sql,model_path)
lines = pd.concat([lines,orifices])

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Valve WHERE Active = 1"
valves = sql_to_df(sql,model_path)
lines = pd.concat([lines,valves])

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], crestlevel AS Outlet_Level FROM msm_Weir WHERE Active = 1"
weirs = sql_to_df(sql,model_path)
lines = pd.concat([lines,weirs])

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], startlevel AS Outlet_Level FROM msm_Pump WHERE Active = 1"
pumps = sql_to_df(sql,model_path)
lines = pd.concat([lines,pumps])

lines['Outlet_Level'].fillna(-9999, inplace=True)

sql = "SELECT muid, acronym, assetname FROM msm_Node WHERE active = 1"
node_id_df = sql_to_df(sql,model_path)
node_id_df = node_id_df[(node_id_df.assetname.str[:2]=='MH') & (node_id_df.assetname.str.len() > 2) & (node_id_df.acronym.notna())]
node_id_df.rename(columns={'muid':'Node'},inplace=True)
node_id_df['ID'] = node_id_df.acronym + '_' + node_id_df.assetname
node_id_df = node_id_df[['Node','ID']]

In [9]:
#Permanent cell 7
#Trace the model

accumulated_catchment_set = set()
accumulated_node_set = set()

for index1, row1 in catchments.iterrows():
    catchment = row1['Catchment']
    nodes = [row1['Connected_Node']]
    start_node = row1['Connected_Node']
    steps = 0
    
    accumulated_catchment_set.add((start_node,catchment))
        
    while steps <= max_steps:
        steps += 1
        downstream_df = lines[lines['From'].isin(nodes)]

        if len(downstream_df) > 0:
            nodes = list(downstream_df.To.unique())

            nodes = [node for node in nodes if len(node)>0]
            for node in nodes:
                accumulated_catchment_set.add((node,catchment))       
        else:
            break
        if steps == max_steps:
            raise ValueError("Maximum steps were reached, indicating a loop. Start catchment is '" + catchment + "'")
           
        accumulated_catchment_set.add((node,catchment))
        
accumulation_df = pd.DataFrame(accumulated_catchment_set,columns=['Node','Catchment'])
accumulation_df = pd.merge(accumulation_df,node_id_df,how='inner',on=['Node'])
data = {
    ('GENERAL INFO', 'CATCHMENT'): accumulation_df.Catchment,
    ('GENERAL INFO', 'NODE'): accumulation_df.Node,
    ('GENERAL INFO', 'ID'): accumulation_df.ID,
}

# Create a DataFrame with MultiIndex columns
accumulation_df = pd.DataFrame(data)



In [10]:
#Permanent cell 8
#Calculate RAWN

catchments = list(pop_df.Catchment.unique())

catchment_df = df_template.copy()
for catchment in catchments:
    for year in years:
        key = model + '@' + catchment + '@' + str(year)
        catchment_df.loc[key,('GENERAL INFO','TYPE')] = 'UNKNOWN'
        catchment_df.loc[key,('GENERAL INFO','CATCHMENT')] = catchment
        catchment_df.loc[key,('GENERAL INFO','YEAR')] = year
        catchment_df.loc[key,('GENERAL INFO','LOCATION')] = model
        for area_col_dict_key in area_col_dict:
            catchment_df.loc[key,(header_dict[area_col_dict_key][0],'AREA (Ha)')] = pop_df.loc[key,area_col_dict[area_col_dict_key]]
        catchment_df.loc[key,('RESIDENTIAL','POPULATION')] = pop_df.loc[key,'Population']
        san_flow = 0
        adwf = 0
        for avg_calc_dict_key in avg_calc_dict:
            input1 = catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][1])]
            input2 = per_unit_dict[avg_calc_dict_key]
            avg_flow = input1 * input2 / 86400
            adwf += avg_flow
            if avg_calc_dict_key not in ['infl','infi']:
                san_flow += avg_flow
            catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][2])] = avg_flow
        catchment_df.loc[key,('FLOWS','AVG. SAN. FLOW (L/s)')] = san_flow
        catchment_df.loc[key,('FLOWS','ADWF (L/s)')] = adwf

        
catchment_node_df = accumulation_df.merge(catchment_df,on=[('GENERAL INFO','CATCHMENT')],how='inner')
node_df = catchment_node_df.copy()
node_df.drop(columns=[('GENERAL INFO','CATCHMENT')],inplace=True)
node_df = node_df.groupby([('GENERAL INFO','NODE'),('GENERAL INFO','TYPE'),('GENERAL INFO','YEAR'),('GENERAL INFO','LOCATION'),('GENERAL INFO','ID')]).sum()
node_df.reset_index(inplace=True)
node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (node_df[('RESIDENTIAL','POPULATION')] / 1000) ** 0.5)) * node_df[('RESIDENTIAL','AVG. FLOW (L/s)')]
node_df[('COMMERCIAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (per_unit_dict['com'] * node_df[('COMMERCIAL','AREA (Ha)')]/(per_unit_dict['res'] * 1000)) ** 0.5))*node_df[('COMMERCIAL','AVG. FLOW (L/s)')]*0.8
node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (per_unit_dict['inst'] * node_df[('INSTITUTIONAL','AREA (Ha)')] / (per_unit_dict['res'] * 1000)) ** 0.5)) * node_df[('INSTITUTIONAL','AVG. FLOW (L/s)')]

mask = node_df[('INDUSTRIAL', 'AREA (Ha)')] != 0 #Avoid error from log(0)
node_df.loc[mask, ('INDUSTRIAL', 'PEAK FLOW (L/s)')] = (
    0.8 * (1 + 14 / (4 + (node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] * per_unit_dict['ind'] / (per_unit_dict['res'] * 1000)) ** 0.5)) *
    np.where(
        node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] < 121,
        1.7,
        2.505 - 0.1673 * np.log(node_df[('INDUSTRIAL', 'AREA (Ha)')][mask])
    ) * node_df[('INDUSTRIAL', 'AVG. FLOW (L/s)')][mask]
)

node_df[('FLOWS','PWWF (L/s)')] = (
    node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] +
    node_df[('COMMERCIAL','PEAK FLOW (L/s)')] +
    node_df[('INDUSTRIAL','PEAK FLOW (L/s)')] +
    node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] +
    node_df[('INFLOW / INFILTRATION','INFLOW (L/s)')] +
    node_df[('INFLOW / INFILTRATION','INFILTRATION (L/s)')]
)

# excel_folder = output_folder + '\\Excel'
# if not os.path.isdir(excel_folder): os.makedirs(excel_folder) 
# for id in node_df[('GENERAL INFO','ID')].unique():    
#     node_single_df = node_df[node_df[('GENERAL INFO','ID')]==id]
#     id = id.replace('/','-') if '/' in id else id
#     node_single_df.to_excel(excel_folder + '\\' + id + '.xlsx')


In [11]:
#Permanent cell 9
#Import GIS from the model
if run_import:

    out_path = output_folder + '\\' + gdb_name

    if not os.path.isdir(out_path):
        arcpy.management.CreateFileGDB(output_folder, gdb_name)

    arcpy.env.workspace = out_path
    sr = arcpy.SpatialReference(26910)

    layers = ['msm_CatchCon','msm_Catchment','msm_Link','msm_Node','msm_Pump','msm_Weir','msm_Orifice','msm_Valve']

    for layer in layers:
        print(layer)
    #     arcpy.conversion.FeatureClassToFeatureClass(model_path + '\\' + layer, out_path, layer)

        arcpy.management.MakeFeatureLayer(model_path + '\\' + layer, "temp_layer", "Active = 1")

        if arcpy.Exists(layer):
            arcpy.management.DeleteFeatures(layer)
            arcpy.management.Append("temp_layer", layer, "NO_TEST")
        else:    
            arcpy.conversion.FeatureClassToFeatureClass("temp_layer", out_path, layer)
            if layer == 'msm_Catchment':
                arcpy.management.AddField('msm_catchment', "Drains_To", "TEXT")

        arcpy.management.Delete("temp_layer")
        arcpy.DefineProjection_management(layer, sr)






In [12]:
#Permanent cell 10
#Create merge_df,minimizing the number of computation heavy dissolves that need to be done.

merge_set = set()

rank_df = accumulation_df[[('GENERAL INFO','NODE'),('GENERAL INFO','CATCHMENT')]].groupby([('GENERAL INFO','NODE')]).count()

# rank_df.reset_index(inplace=True)
rank_df.columns = ['Catchment_Count']
max_catchments = max(rank_df.Catchment_Count)
rank_df.sort_values(by=['Catchment_Count'],inplace=True)
# rank_df.reset_index(inplace=True)

catchment_list = []
merge_set = set()
for index, row in rank_df.iterrows():

    catchments = list(accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==index][('GENERAL INFO','CATCHMENT')].unique())
    catchments = tuple(sorted(catchments))
#     rank_df.loc[index,'Catchments'] = catchments
    catchment_list.append(catchments)
    merge_set.add(catchments)
    
    
rank_df['Catchments'] = catchment_list
rank_df['Node'] = rank_df.index
print(len(merge_set))

merge_list = []
for i, catchments in enumerate(merge_set):
    merge_id = 'Merge_ID_' + str(i)
    merge_list.append([merge_id,catchments])
    
merge_df = pd.DataFrame(merge_list,columns=['Merge_ID','Catchments'])
merge_df['Catchment_Count'] = merge_df['Catchments'].apply(len)
merge_df.sort_values(by=['Catchment_Count'],ascending=False,inplace=True)
merge_df.reset_index(inplace=True,drop=True)

simpler_merge = []
for index1, row1 in merge_df.iterrows():
    catchments1 = list(row1['Catchments'])
    for index2, row2 in merge_df[index1+1:].iterrows():
        catchments2 = row2['Catchments']
        
        if len(catchments1) >= len(catchments2):
            if all(item in catchments1 for item in catchments2):
                catchments1 = [catchment for catchment in catchments1 if catchment not in catchments2]
                catchments1.append(row2['Merge_ID'])
    simpler_merge.append(catchments1)
               
merge_df['To_Dissolve'] = simpler_merge
merge_df.sort_values(by=['Catchment_Count'],inplace=True)
merge_df.reset_index(inplace=True,drop=True)

rank_df = pd.merge(rank_df,merge_df[['Merge_ID','Catchments']], on=['Catchments'],how='inner')
rank_df.set_index('Node',inplace=True)
    
print(max_catchments)

# for index1, row1 in merge_df.iterrows():
#     print

rank_df

103
105


,Catchment_Count,Catchments,Merge_ID
Node,,,
7311,1,"(2169,)",Merge_ID_70
7312,1,"(2169,)",Merge_ID_70
7310,1,"(2169,)",Merge_ID_70
7112,1,"(2169,)",Merge_ID_70
6786,1,"(2122,)",Merge_ID_98
...,...,...,...
9766,104,"(10108, 10110, 2059, 2060, 2061, 2062, 2063, 2...",Merge_ID_87
9764,104,"(10108, 10110, 2059, 2060, 2061, 2062, 2063, 2...",Merge_ID_87
9763,104,"(10108, 10110, 2059, 2060, 2061, 2062, 2063, 2...",Merge_ID_87


In [26]:
#Permanent cell 11
#Run dissolve
out_path = output_folder + '\\' + gdb_name
arcpy.env.workspace = out_path  
arcpy.env.addOutputsToMap = False
if run_dissolve:
    if arcpy.Exists(gdb_name_dissolve):
        arcpy.management.Delete(gdb_path)
    arcpy.management.CreateFileGDB(output_folder, gdb_name_dissolve)
    dissolve_path = output_folder + '\\' + gdb_name_dissolve
    arcpy.conversion.FeatureClassToFeatureClass('msm_Catchment', dissolve_path, 'Node_Catchment')
#     arcpy.env.workspace = dissolve_path
    arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Drains_To", "TEXT")
    arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Merge_ID", "TEXT")
    arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Merge_ID_Temp", "TEXT")
    arcpy.management.CalculateField(dissolve_path + '\\Node_Catchment', "Merge_ID", "!muid!", "PYTHON3")
    for index, row in merge_df.iterrows():
        arcpy.management.CalculateField(dissolve_path + '\\Node_Catchment', "Merge_ID_Temp", "''", "PYTHON3")
        print(f'Dissolving for {row["Merge_ID"]}, {index} of {max(merge_df.index)} at time {datetime.datetime.now()}')
        if row['Catchment_Count'] == 1:
            arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
            where_clause = f"muid = '{row['To_Dissolve'][0]}'"
            arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
            arcpy.management.CalculateField("temp_layer", "Merge_ID", f"'{row['Merge_ID']}'", "PYTHON3")
            # Clean up by removing the feature layer
            arcpy.management.Delete("temp_layer")
        else:
            arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
            catchments = row['To_Dissolve']
            catchments_sql = ', '.join([f"'{muid}'" for muid in catchments])
            where_clause = f"Merge_ID in ({catchments_sql})"
            arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
            arcpy.management.CalculateField("temp_layer", "Merge_ID_Temp", f"'{row['Merge_ID']}'", "PYTHON3")
            arcpy.management.Dissolve("temp_layer",dissolve_path + '\\Dissolve_Temp', "Merge_ID_Temp", "", "MULTI_PART")
            arcpy.management.Delete("temp_layer")
            arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Merge_ID", f"'{row['Merge_ID']}'", "PYTHON3")
            rank_df.head(1)

            nodes = list(rank_df[rank_df.Merge_ID==row["Merge_ID"]].index)
            for node in nodes:
                arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Drains_To", f"'{node}'", "PYTHON3")
                arcpy.management.Append(dissolve_path + '\\Dissolve_Temp', dissolve_path + '\\Node_Catchment', "NO_TEST")
            
            arcpy.management.Delete("temp_layer")
            arcpy.management.Delete(dissolve_path + '\\Dissolve_Temp')
            

# # Use CalculateField to update the Drains_To field
# arcpy.management.CalculateField("catchment_layer", "Drains_To", f"'{node}'", "PYTHON3")

#Delete the features without a Drains_To
arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
where_clause = f"Drains_To IS NULL"
arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
arcpy.management.DeleteFeatures("temp_layer")  
arcpy.management.Delete("temp_layer")

#Append the features into the official Node_Catchment layer
arcpy.management.MakeFeatureLayer("Node_Catchment", "Temp_Layer")
arcpy.management.DeleteFeatures("Temp_Layer")
arcpy.management.Append(dissolve_path + '\\Node_Catchment', "Temp_Layer", "NO_TEST")
arcpy.management.Delete("temp_layer")

          
print('Done')            
    


Dissolving for Merge_ID_70, 0 of 102 at time 2024-05-30 15:32:32.724306
Dissolving for Merge_ID_98, 1 of 102 at time 2024-05-30 15:32:35.875191
Dissolving for Merge_ID_38, 2 of 102 at time 2024-05-30 15:32:39.108152
Dissolving for Merge_ID_67, 3 of 102 at time 2024-05-30 15:32:42.195980
Dissolving for Merge_ID_35, 4 of 102 at time 2024-05-30 15:33:09.240746
Dissolving for Merge_ID_83, 5 of 102 at time 2024-05-30 15:33:34.090502
Dissolving for Merge_ID_71, 6 of 102 at time 2024-05-30 15:33:50.931924
Dissolving for Merge_ID_12, 7 of 102 at time 2024-05-30 15:34:03.182142
Dissolving for Merge_ID_76, 8 of 102 at time 2024-05-30 15:34:15.740643
Dissolving for Merge_ID_97, 9 of 102 at time 2024-05-30 15:34:28.495837
Dissolving for Merge_ID_61, 10 of 102 at time 2024-05-30 15:35:18.761868
Dissolving for Merge_ID_6, 11 of 102 at time 2024-05-30 15:35:32.365325
Dissolving for Merge_ID_19, 12 of 102 at time 2024-05-30 15:35:59.471147
Dissolving for Merge_ID_1, 13 of 102 at time 2024-05-30 15:36:

In [ ]:
# merge_ids = set()
# for index, row in rank_df.iterrows():
#     print(index)
#     merge_id = row['Merge_ID']
#     arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
#     where_clause = f"Merge_ID = '{merge_id}'"
#     arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
    
#     if merge_id in merge_ids:
#         arcpy.conversion.FeatureClassToFeatureClass("temp_layer", dissolve_path, 'Dissolve_Temp')
#         arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Drains_To", f'"{index}"', "PYTHON3")
#         arcpy.management.Append(dissolve_path + '\\Dissolve_Temp', dissolve_path + '\\Node_Catchment', "NO_TEST")
#         arcpy.management.Delete(dissolve_path + '\\Dissolve_Temp')
#     else:
#         arcpy.management.CalculateField("temp_layer", "Drains_To", f'"{index}"', "PYTHON3")
        
#     arcpy.management.Delete("temp_layer")
#     merge_ids.add(merge_id)
    

7311
7312
7310
7112
6786
6787
6788
6789
6790
9670
9671
6791
6792
6794
6795
6793
9746
9747
9744
9745
9673
9672
7313
9942
7314
7318
7319
7479
7480
7481
7317
7316
7315
7114
7113
7189
7190
7141
7140
7139
7142
7143
9783
9757
7191
9756
9613
9614
7144
7145
7146
7147
7148
9483
10124
9758
9728
9727
9726
9615
7149
7150
7151
7152
9748
9606
9605
9604
9603
9602
9601
9600
9599
9598
9729
9730
9485
9486
9484
9617
9616
9984
7154
7155
7156
7157
7158
7159
7160
7161
7162
7153
9805
9739
9731
9732
9733
9734
9735
9736
9738
9737
9789
9777
9742
9741
9740
9778
9954
9755
9753
9754
9607
9609
9610
9611
9752
9608
9750
9751
9749
9612
9489
9487
9488
7163
7164
9792
9790
9804
10004
10003
9787
9780
9779
9781
7115
9493
9492
9491
9490
7116
7117
7167
7165
7166
7168
7169
10482
10484
10497
10499
10501
10504
10498
9550
9760
9759
9494
9495
9496
9497
9498
9500
9499
7172
7173
7174
7171
7170
9551
9554
9553
9552
7176
7175
9556
9555
9557
7119
6496
9501
9502
9503
10020
7177
9563
9562
9561
9560
9559
9558
9786
9784
9785
10001
10002
95

<Result 'true'>

In [2]:
merge_df.head(5)

NameError: name 'merge_df' is not defined

In [14]:
#Obsolete cell
# #Dissolve catchments
# arcpy.env.addOutputsToMap = False
# if run_dissolve:
#     arcpy.management.CreateFileGDB(output_folder, gdb_name_dissolve)
#     dissolve_path = output_folder + '\\' + gdb_name_dissolve
#     arcpy.conversion.FeatureClassToFeatureClass('msm_Catchment', dissolve_path, 'msm_Catchment')
#     nodes = list(accumulation_df[('GENERAL INFO','NODE')].unique())
#     for i, node in enumerate(nodes):
#         print('Dissolving for node ' + str(i) + ' of ' + str(len(nodes)) + ' at time ' + str(datetime.datetime.now()))
#         catchment_df = accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==node]
#         catchments = list(catchment_df[('GENERAL INFO','CATCHMENT')].unique())
#         arcpy.management.CalculateField(dissolve_path + '\\msm_Catchment', "Drains_To", "''", "PYTHON3")
#         with arcpy.da.UpdateCursor(dissolve_path + '\\msm_catchment', ['muid', 'Drains_To']) as cursor:
#             for row in cursor:
#                 if row[0] in catchments:
#                     row[1] = node
#                     cursor.updateRow(row)

#         query = "Drains_To = 'Test'"
#         arcpy.management.MakeFeatureLayer(dissolve_path + '\\msm_catchment', "temp_layer", "Drains_To = '" + node + "'")
#         dissolve_output = dissolve_path + '\\msm_Catchment_Dissolve_Single'
#         arcpy.management.Dissolve("temp_layer", dissolve_output, "Drains_To", "", "MULTI_PART")
#         arcpy.management.Delete("temp_layer")

#         arcpy.conversion.FeatureClassToFeatureClass(dissolve_path + '\\msm_Catchment_Dissolve_Single', dissolve_path, 'Node_Catchment_' + node)



In [16]:
#Obsolete cell
#Append individual dissolved catchments to one layer.

# if run_dissolve_append:
#     nodes = list(accumulation_df[('GENERAL INFO','NODE')].unique())
#     for i, node in enumerate(nodes):    
#         print('Appending for node ' + str(i) + ' of ' + str(len(nodes)) + ' at time ' + str(datetime.datetime.now()))
#         if i == 0:
#             arcpy.conversion.FeatureClassToFeatureClass(dissolve_path + '\\Node_Catchment_' + node, out_path, 'Node_Catchment')
#         else:
#             arcpy.management.Append(dissolve_path + '\\Node_Catchment_' + node, "Node_Catchment", "NO_TEST")


In [149]:
#Permanent cell 12
#Export jpgs
if run_jpg:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    project_path = aprx.filePath

    jpg_folder = output_folder + r'\HTML\Maps_And_CSS'
    if not os.path.isdir(jpg_folder): os.makedirs(jpg_folder) 


    # project_directory = os.path.dirname(project_path)

    layouts = aprx.listLayouts()
    export_fails = []

    for layout in layouts:

        if layout.mapSeries is not None:
            map_series = layout.mapSeries
            # Loop through all pages in the map series
            for page_number in range(1, map_series.pageCount + 1):
                map_series.currentPageNumber = page_number
                output_filename = os.path.join(jpg_folder, f"{map_series.pageRow.Drains_To}.jpg")
                try:
                    layout.exportToJPEG(output_filename, resolution=300)
                except:
                    print(f'WARNING! {map_series.pageRow.Drains_To} could not be made')
                    export_fails.append(map_series.pageRow.Drains_To)
                print (f'Printing jpg {page_number} of {map_series.pageCount} at time {datetime.datetime.now()}')

    print(f'the following pages failed: {export_fails}')            
    print("Export complete.")




Printing jpg 1 of 458 at time 2024-05-31 14:28:45.950136
Printing jpg 2 of 458 at time 2024-05-31 14:28:48.131127
Printing jpg 3 of 458 at time 2024-05-31 14:28:50.248059
Printing jpg 4 of 458 at time 2024-05-31 14:28:52.531143
Printing jpg 5 of 458 at time 2024-05-31 14:28:54.714136
Printing jpg 6 of 458 at time 2024-05-31 14:28:57.064281
Printing jpg 7 of 458 at time 2024-05-31 14:28:59.914883
Printing jpg 8 of 458 at time 2024-05-31 14:29:02.147922
Printing jpg 9 of 458 at time 2024-05-31 14:29:04.347930
Printing jpg 10 of 458 at time 2024-05-31 14:29:06.818185
Printing jpg 11 of 458 at time 2024-05-31 14:29:09.186852
Printing jpg 12 of 458 at time 2024-05-31 14:29:11.483949
Printing jpg 13 of 458 at time 2024-05-31 14:29:13.833093
Printing jpg 14 of 458 at time 2024-05-31 14:29:16.415451
Printing jpg 15 of 458 at time 2024-05-31 14:29:18.631474
Printing jpg 16 of 458 at time 2024-05-31 14:29:20.826477
Printing jpg 17 of 458 at time 2024-05-31 14:29:22.947413
Printing jpg 18 of 458 

Printing jpg 141 of 458 at time 2024-05-31 14:34:03.080130
Printing jpg 142 of 458 at time 2024-05-31 14:34:05.247108
Printing jpg 143 of 458 at time 2024-05-31 14:34:07.263949
Printing jpg 144 of 458 at time 2024-05-31 14:34:09.246759
Printing jpg 145 of 458 at time 2024-05-31 14:34:11.830117
Printing jpg 146 of 458 at time 2024-05-31 14:34:13.947049
Printing jpg 147 of 458 at time 2024-05-31 14:34:16.263164
Printing jpg 148 of 458 at time 2024-05-31 14:34:18.246975
Printing jpg 149 of 458 at time 2024-05-31 14:34:20.606128
Printing jpg 150 of 458 at time 2024-05-31 14:34:22.963280
Printing jpg 151 of 458 at time 2024-05-31 14:34:25.330441
Printing jpg 152 of 458 at time 2024-05-31 14:34:27.555472
Printing jpg 153 of 458 at time 2024-05-31 14:34:29.813533
Printing jpg 154 of 458 at time 2024-05-31 14:34:31.597161
Printing jpg 155 of 458 at time 2024-05-31 14:34:33.596987
Printing jpg 156 of 458 at time 2024-05-31 14:34:35.811008
Printing jpg 157 of 458 at time 2024-05-31 14:34:37.8808

Printing jpg 279 of 458 at time 2024-05-31 14:39:22.813631
Printing jpg 280 of 458 at time 2024-05-31 14:39:24.873510
Printing jpg 281 of 458 at time 2024-05-31 14:39:27.046494
Printing jpg 282 of 458 at time 2024-05-31 14:39:29.232489
Printing jpg 283 of 458 at time 2024-05-31 14:39:31.263343
Printing jpg 284 of 458 at time 2024-05-31 14:39:33.424316
Printing jpg 285 of 458 at time 2024-05-31 14:39:35.447162
Printing jpg 286 of 458 at time 2024-05-31 14:39:37.563093
Printing jpg 287 of 458 at time 2024-05-31 14:39:39.714056
Printing jpg 288 of 458 at time 2024-05-31 14:39:41.687858
Printing jpg 289 of 458 at time 2024-05-31 14:39:43.713707
Printing jpg 290 of 458 at time 2024-05-31 14:39:45.695516
Printing jpg 291 of 458 at time 2024-05-31 14:39:47.580236
Printing jpg 292 of 458 at time 2024-05-31 14:39:49.723192
Printing jpg 293 of 458 at time 2024-05-31 14:39:51.913191
Printing jpg 294 of 458 at time 2024-05-31 14:39:54.030637
Printing jpg 295 of 458 at time 2024-05-31 14:39:56.0965

Printing jpg 417 of 458 at time 2024-05-31 14:44:56.834775
Printing jpg 418 of 458 at time 2024-05-31 14:44:59.502210
Printing jpg 419 of 458 at time 2024-05-31 14:45:02.053538
Printing jpg 420 of 458 at time 2024-05-31 14:45:04.492764
Printing jpg 421 of 458 at time 2024-05-31 14:45:07.092137
Printing jpg 422 of 458 at time 2024-05-31 14:45:09.472309
Printing jpg 423 of 458 at time 2024-05-31 14:45:11.892518
Printing jpg 424 of 458 at time 2024-05-31 14:45:14.312230
Printing jpg 425 of 458 at time 2024-05-31 14:45:16.429162
Printing jpg 426 of 458 at time 2024-05-31 14:45:18.646185
Printing jpg 427 of 458 at time 2024-05-31 14:45:20.712070
Printing jpg 428 of 458 at time 2024-05-31 14:45:22.863034
Printing jpg 429 of 458 at time 2024-05-31 14:45:24.978965
Printing jpg 430 of 458 at time 2024-05-31 14:45:27.614370
Printing jpg 431 of 458 at time 2024-05-31 14:45:30.446955
Printing jpg 432 of 458 at time 2024-05-31 14:45:32.916209
Printing jpg 433 of 458 at time 2024-05-31 14:45:35.5295

In [18]:
#Permanent cell 13
#Create HTMLs

if run_html:
    shutil.copy2('style.css', html_folder + '\\style.css')
    shutil.copy2('script.js', html_folder + '\\script.js')

    for category in categories:
        area_type = category[0]
        area_names = category[1]
        header_start = category[2]

        f = open(html_folder + '\\Population_By_' + area_type + '_' + model_area + '.html', "w")
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write('<script src="script.js"></script>\n')
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write('<!DOCTYPE html>\n')
        f.write('<html>\n')
        f.write('<head>\n')
        f.write('<meta charset="utf-8">\n')
        f.write('</head>\n')
        f.write('<body>\n\n')

        f.write('<div class="tab">\n')
        for area_name in area_names:
            tab = area_name

        #     color = ps_dict[first_year]
        #     bg_color = color_dict[color][0]
        #     text_color = color_dict[color][1]

            f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
        f.write('</div>\n')

        pop_df = pop_dfss[0][2]

        for area_name in area_names:

            area_df = pop_df[pop_df[area_type]==area_name]
            area_df = area_df[['Year','Population']].groupby(['Year']).sum()

            f.write('<div id="' + area_name + '" class="tabcontent">\n') 
            f.write('<h1>' + area_name + '</h1>\n')

            f.write('<div class="sidenav">\n')

            f.write('<table style=\'width: 90%;\'>\n')
            f.write('<tr>\n')
            f.write('<th>Year</th>\n')
            f.write('<th>Population</th>\n')
            f.write('</tr>\n')

            for index, row in area_df.iterrows():
                f.write('<tr>\n')
                f.write('<td>'+ str(index) + '</td>\n')
                population_with_separator = f"{int(row['Population']):,}"
                f.write('<td>'+ population_with_separator + '</td>\n')

                f.write('</tr>\n')
            f.write('</table>\n')

            for i in range(4):
                f.write('<h1 style="color: white">End of tables</h1>\n')#Invisible, just to enable scroll to table bottoms

            f.write('</div>\n') #end sidenav


            f.write('<div class="main">\n')

            fig = go.Figure()


            fig.add_trace(go.Scatter(x=area_df.index, 
                                         y = area_df.Population, 
                                         mode='lines',name=pop_dfss[0][0],line=dict(width=5)))

            for pop_dfs in pop_dfss[1:]:
                pop_df_past = pop_dfs[2]
                area_df = pop_df_past[pop_df_past[area_type]==area_name]
                area_df = area_df[['Year','Population']].groupby(['Year']).sum()
                fig.add_trace(go.Scatter(x=area_df.index, 
                                         y = area_df.Population, 
                                         mode='lines',name=pop_dfs[0],line=dict(width=2)))

            fig.update_layout(
                title=header_start + area_name,
                autosize=False,
                width = 1500,
                height=850,
                margin=dict(
                    l=50,
                    r=50,
                    b=50,
                    t=50,
                    pad=4
                    ),
                    yaxis_title = 'Population'
                )

            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

            f.write('</div>\n') #end div main  

            f.write('</div>\n')  #end div tab   

            f.write('</body>\n')
        f.write('</html>\n')
        f.close()



In [19]:
# import pandas as pd

# # Sample DataFrame
# data = {
#     'Catchment': [1, 2, 3],
#     'Node': [4, 5, 6]

# }
# accumulation_df = pd.DataFrame(data)

# # Create a MultiIndex with the existing columns
# existing_columns_multiindex = pd.MultiIndex.from_tuples([
#     ('GENERAL INFO', 'Catchment'),  # Header with no subheaders
#     ('GENERAL INFO', 'Node'),  # Header with no subheaders
# ])

# # Create a MultiIndex with the upper level 'GENERAL INFO'
# upper_level = [('GENERAL INFO', '')] * len(existing_columns_multiindex)

# # Concatenate the upper level and the existing columns MultiIndex
# new_columns_multiindex = pd.MultiIndex.from_tuples(list(zip(upper_level, existing_columns_multiindex)))

# # Assign the new MultiIndex to the DataFrame columns
# accumulation_df.columns = new_columns_multiindex

# accumulation_df


In [79]:
# merge_set = set()
# for node in accumulation_df[('GENERAL INFO','NODE')].unique():
#     catchments = list(accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==node][('GENERAL INFO','CATCHMENT')].unique())
#     catchments = tuple(sorted(catchments))
#     merge_set.add(catchments)
# print(len(merge_set))

103


In [14]:
# cou = 0

# simpler_merge = []
# for index1, row1 in merge_df.iterrows():
#     catchments1 = list(row1['Catchments'])
#     for index2, row2 in merge_df[index1+1:].iterrows():
#         catchments2 = row2['Catchments']
        
#         if len(catchments1) >= len(catchments2):
#             if all(item in catchments1 for item in catchments2):
#                 catchments1 = [catchment for catchment in catchments1 if catchment not in catchments2]
#                 catchments1.append(row2['Merge_ID'])
#     simpler_merge.append(catchments1)
               
# merge_df['To_Dissolve'] = simpler_merge
# merge_df.sort_values(by=['Catchment_Count'],inplace=True)
# merge_df.reset_index(inplace=True,drop=True)
# merge_df


,Merge_ID,Catchments,Catchment_Count,To_Dissolve
0,Merge_ID_61,"(2169,)",1,[2169]
1,Merge_ID_25,"(2052,)",1,[2052]
2,Merge_ID_80,"(2122,)",1,[2122]
3,Merge_ID_24,"(2119, 2122)",2,"[2119, Merge_ID_80]"
4,Merge_ID_9,"(2050, 2052)",2,"[2050, Merge_ID_25]"
...,...,...,...,...
98,Merge_ID_6,"(2059, 2062, 2063, 2064, 2066, 2068, 2069, 207...",95,"[Merge_ID_81, Merge_ID_90]"
99,Merge_ID_75,"(2059, 2061, 2062, 2063, 2064, 2065, 2066, 206...",98,"[2061, 2065, 2067, Merge_ID_6]"
100,Merge_ID_37,"(2059, 2060, 2061, 2062, 2063, 2064, 2065, 206...",99,"[2060, Merge_ID_75]"
101,Merge_ID_20,"(10108, 10110, 2059, 2060, 2061, 2062, 2063, 2...",104,"[10108, 10110, 2070, 2075, 2089, Merge_ID_37]"


In [134]:
help(cell.border)

Help on StyleProxy in module openpyxl.styles.proxy object:

class StyleProxy(builtins.object)
 |  StyleProxy(target)
 |  
 |  Proxy formatting objects so that they cannot be altered
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |      Add proxied object to another instance and return the combined object
 |  
 |  __copy__(self)
 |      Return a copy of the proxied object.
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __getattr__(self, attr)
 |  
 |  __init__(self, target)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __ne__(self, other)
 |      Return self!=value.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __setattr__(self, attr, value)
 |      Implement setattr(self, name, value).
 |  
 |  copy(self, **kw)
 |      Return a copy of the proxied object. Keyword args will be passed through
 |      
 |      .. note::
 |          Deprecated: Use copy(obj) or cell.obj = cell.obj + other
 |  
 |  -----------

In [41]:
help(df1.to_excel)

Help on method to_excel in module pandas.core.generic:

to_excel(excel_writer, sheet_name: 'str' = 'Sheet1', na_rep: 'str' = '', float_format: 'str | None' = None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True, freeze_panes=None, storage_options: 'StorageOptions' = None) -> 'None' method of pandas.core.frame.DataFrame instance
    Write object to an Excel sheet.
    
    To write a single object to an Excel .xlsx file it is only necessary to
    specify a target file name. To write to multiple sheets it is necessary to
    create an `ExcelWriter` object with a target file name, and specify a sheet
    in the file to write to.
    
    Multiple sheets may be written to by specifying unique `sheet_name`.
    With all data written to the file it is necessary to save the changes.
    Note that creating an `ExcelWriter` object with a file name that already
    exists will result in t

In [89]:
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, Alignment, PatternFill
node_single_df

In [155]:
node_single_df.reset_index(drop=True,inplace=True)

In [148]:
hex_blue = "ADD8E6"
hex_yellow = "FFFACD"
border_style = Side(style='thin', color='000000')
border = Border(top=border_style, bottom=border_style, left=border_style, right=border_style)
border_style_none = Side(style=None, color='000000')
border_none = Border(top=border_style_none, bottom=border_style_none, left=border_style_none, right=border_style_none)

excel_folder = output_folder + '\\Excel'
if not os.path.isdir(excel_folder): os.makedirs(excel_folder) 
for id in node_df[('GENERAL INFO','ID')].unique():    
    node_single_df = node_df[node_df[('GENERAL INFO','ID')]==id].copy()
    node_single_df.reset_index(drop=True,inplace=True)

    sheetpath = excel_folder + "\\" + id + ".xlsx"
    startrow = 13
    with pd.ExcelWriter(sheetpath) as writer:
        node_single_df.to_excel(writer, sheet_name="Sheet1",startrow=startrow)
        info_df.to_excel(writer, sheet_name="Sheet1",startrow=1,startcol=2)

        workbook = writer.book
        workbook.create_sheet("Map")

    workbook = load_workbook(sheetpath)    
    sheet1 = workbook[id]

    #Format infobox
    merged_range = sheet1.merged_cells
    for col in sheet1.iter_cols(min_col=3,max_col=5):
        for cell in col[1:2]:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")
        for cell in col[2:7]:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.fill = PatternFill(start_color=hex_yellow, end_color=hex_yellow, fill_type="solid")
            cell.border = border
    sheet1.column_dimensions['C'].width = 22 
    sheet1.column_dimensions['D'].width = 11 

    #Remove index
    for row in sheet1.iter_rows():
        for cell in row[:1]:
            cell.value = ''
            cell.border = border_none
    #Format main table header rows
    merged_range = sheet1.merged_cells
    for col in sheet1.iter_cols(min_col=2):
        for cell in col[startrow+1:startrow+2]:
            cell.alignment = Alignment(horizontal="center", vertical="center",wrap_text=True)
            cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")
        for cell in col[startrow:startrow+1]:
            if cell.coordinate in sheet1.merged_cells:
                cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")        

    sheet1.column_dimensions['C'].width = 22 
    sheet1.column_dimensions['D'].width = 11   
    sheet1.column_dimensions['F'].width = 13
    sheet1.column_dimensions['H'].width = 13  
    sheet1.column_dimensions['V'].width = 13     

    sheet = workbook["Map"]

    # Add an image to the sheet
    img_path = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\Rawn_Tool\Output\HTML\Maps_And_CSS\9908.jpg"  # Replace with the path to your image
    img = Image(img_path)

    # Set the position for the image (e.g., 'B2' for cell B2)
    sheet.add_image(img, 'B2')

    workbook.save(sheetpath)




    

In [147]:
dir(merged_cell)

['_CellRange__superset', '__add__', '__and__', '__attrs__', '__class__', '__contains__', '__copy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__elements__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__namespaced__', '__ne__', '__nested__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_title', '_get_borders', 'bottom', 'bounds', 'cells', 'cols', 'coord', 'expand', 'fill', 'format', 'from_tree', 'idx_base', 'intersection', 'isdisjoint', 'issubset', 'issuperset', 'left', 'max_col', 'max_row', 'min_col', 'min_row', 'namespace', 'right', 'rows', 'shift', 'shrink', 'size', 'start_cell', 'tagname', 'title', 'to_tree', 'top', 'union', 'ws']

In [112]:
sheet1.merged_cells

<MultiCellRange [B14:F14 G14:J14 K14:M14 N14:P14 Q14:S14 T14:V14 W14:Y14]>

In [86]:
dir(cell)

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_bind_value', '_comment', '_hyperlink', '_style', '_value', 'alignment', 'base_date', 'border', 'check_error', 'check_string', 'col_idx', 'column', 'column_letter', 'comment', 'coordinate', 'data_type', 'encoding', 'fill', 'font', 'has_style', 'hyperlink', 'internal_value', 'is_date', 'number_format', 'offset', 'parent', 'pivotButton', 'protection', 'quotePrefix', 'row', 'style', 'style_id', 'value']

In [38]:
help(ExcelWriter)

Help on class ExcelWriter in module pandas.io.excel._base:

class ExcelWriter(builtins.object)
 |  ExcelWriter(path: 'FilePath | WriteExcelBuffer | ExcelWriter', engine: 'str | None' = None, date_format: 'str | None' = None, datetime_format: 'str | None' = None, mode: 'str' = 'w', storage_options: 'StorageOptions' = None, if_sheet_exists: "Literal['error', 'new', 'replace', 'overlay'] | None" = None, engine_kwargs: 'dict | None' = None, **kwargs)
 |  
 |  Class for writing DataFrame objects into excel sheets.
 |  
 |  Default is to use :
 |  * xlwt for xls
 |  * xlsxwriter for xlsx if xlsxwriter is installed otherwise openpyxl
 |  * odf for ods.
 |  See DataFrame.to_excel for typical usage.
 |  
 |  The writer should be used as a context manager. Otherwise, call `close()` to save
 |  and close any opened file handles.
 |  
 |  Parameters
 |  ----------
 |  path : str or typing.BinaryIO
 |      Path to xls or xlsx or ods file.
 |  engine : str (optional)
 |      Engine to use for writin

In [35]:

node_single_df
writer = pd.ExcelWriter('file_name.xlsx', engine='xlsxwriter')
workbook = writer.book

ModuleNotFoundError: No module named 'xlsxwriter'

In [20]:
# import pandas as pd

# # Sample data for the DataFrame
# data = {
#     ('GENERAL INFO', 'CATCHMENT'): accumulation_df.Catchment,
#     ('GENERAL INFO', 'NODE'): accumulation_df.Node,
# }

# # Create a DataFrame with MultiIndex columns
# df = pd.DataFrame(data)

# # Set names for the levels of the MultiIndex
# # df.columns.names = ['Header', 'Subheader']

# df

In [15]:
# import sys

# def get_memory_usage(var):
#     """Returns the memory usage of a variable in bytes."""
#     return sys.getsizeof(var)

# def list_variables_memory():
#     # Get the local and global variables
#     variables = {**globals(), **locals()}
#     total_memory = 0
    
#     # Print the variables and their memory consumption
#     for var_name, var_value in variables.items():
#         # Filter out the built-in variables and functions
#         if not var_name.startswith('__') and not callable(var_value):
#             memory = get_memory_usage(var_value)
#             total_memory += memory
#             print(f"Variable: {var_name}, Type: {type(var_value)}, Memory: {memory} bytes")

#     total_memory_mb = total_memory / (1024 * 1024)  # Convert bytes to megabytes
#     print(f"Total Memory Usage: {total_memory_mb:.2f} MB")
#     return total_memory_mb


# # Call the function to list variables and their memory consumption
# total_memory_usage_mb = list_variables_memory()
